In [1]:
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiPoint, MultiLineString
from shapely.ops import nearest_points
import pandas as pd
import glob
import os

#print(gdf_distritos[["ds_codigo", "ds_nome"]])

In [6]:
gdf_distritos = gpd.read_file(os.path.join(os.pardir, "download", "SIRGAS_SHP_distrito", "SIRGAS_SHP_distrito_polygon.shp"))

In [8]:
gdf_vias = gpd.read_file(os.path.join(os.pardir, "download", "SIRGAS_SHP_logradouronbl", "SIRGAS_SHP_logradouronbl.shp"))

In [9]:
gdf_ferrovias = gpd.read_file(os.path.join(os.pardir, "download", "SIRGAS_SHP_ferroviamdc", "SIRGAS_SHP_ferroviamdc.shp"))

In [10]:
gdf_ferrovias['lg_codlog'] = 'ferrovia'
gdf_vias = gpd.GeoDataFrame(pd.concat([gdf_vias, gdf_ferrovias]).reset_index().drop(columns=['index']))

In [11]:
#gdf_quadras = gpd.read_file('../download/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')

In [12]:
def centroide_por_segmento(linha):
    segmentos = list(map(LineString, zip(linha.coords[:-1], linha.coords[1:])))
    centroides = list(map(lambda x: x.centroid, segmentos))
    return MultiPoint(centroides)

In [13]:
def segmentos(linha):
    segmentos = list(map(LineString, zip(linha.coords[:-1], linha.coords[1:])))
    # centroides = list(map(lambda x: x.centroid, segmentos))
    return MultiLineString(segmentos)

In [20]:
## Para cada distrito
folder = '../download/lotes'
for path in glob.iglob(f"{folder}/*"):
    cod_distrito = path.replace(f"{folder}/SIRGAS_SHP_LOTES_", '')[0:2]
    gdf_distrito = gdf_distritos[gdf_distritos.ds_codigo == str(int(cod_distrito))]

    ## Recorta NBL apenas do Distrito considerando um buffer de 20m
    gdf_viario = gpd.clip(gdf_vias, gdf_distrito.buffer(30))

    ## Remove tipo de logradouro PA
    gdf_viario = gdf_viario.loc[gdf_viario.lg_tipo != 'PA']

    ## Dissolve a NBL por CODLOG
    gdf_viario = gdf_viario.dissolve(by='lg_codlog').reset_index()

    ## Abre o arquivo de lotes do distrito
    gdf_lotes = gpd.read_file(f"zip://{path}!{path.replace(folder, '').replace('.zip', '')[1:]}")

    ## Cria um viário unificado para projetar os pontos mais próximos
    gdf_viario_unified = gdf_viario.unary_union

    ## Remove os lotes do Tipo V que são sobrepostos por viário
    gdf_lotes_v_vias = gdf_lotes[gdf_lotes.lo_tp_lote == 'V'].geometry.intersects(gdf_viario_unified)
    gdf_lotes.drop(gdf_lotes_v_vias[gdf_lotes_v_vias].index, inplace=True)

    ## Remove os lotes do Tipo V que NÃO SÃO sobrepostos por viários e criar um segmento de linha com CODLOG provisório para conectar a testada
    if len(gdf_lotes.loc[gdf_lotes.lo_tp_lote == 'V', ['lo_tp_lote', 'lo_setor', 'lo_quadra', 'lo_lote']].values.tolist()) > 0:
        gdf_viario = gpd.GeoDataFrame(pd.concat([gdf_viario, gpd.GeoDataFrame(pd.Series(gdf_lotes.loc[gdf_lotes.lo_tp_lote == 'V', ['lo_tp_lote', 'lo_setor', 'lo_quadra', 'lo_lote']].values.tolist(), index=gdf_lotes[gdf_lotes.lo_tp_lote == 'V'].index, name='lg_codlog').str.join(''), geometry=gdf_lotes[gdf_lotes.lo_tp_lote == 'V'].buffer(-.5).boundary)]))
        gdf_viario_unified = gdf_viario.unary_union

    ## Removendo lotes tipo V
    gdf_lotes = gdf_lotes[gdf_lotes.lo_tp_lote != 'V']

    ## Remove os lotes do Tipo M e cria um segmento de linha com CODLOG provisório para conectar a testada
    if len(gdf_lotes.loc[gdf_lotes.lo_tp_lote == 'M', ['lo_tp_lote', 'lo_setor', 'lo_quadra', 'lo_lote']].values.tolist()) > 0:
        gdf_viario = gpd.GeoDataFrame(pd.concat([gdf_viario, gpd.GeoDataFrame(pd.Series(gdf_lotes.loc[gdf_lotes.lo_tp_lote == 'M', ['lo_tp_lote', 'lo_setor', 'lo_quadra', 'lo_lote']].values.tolist(), index=gdf_lotes[gdf_lotes.lo_tp_lote == 'M'].index, name='lg_codlog').str.join(''), geometry=gdf_lotes[gdf_lotes.lo_tp_lote == 'M'].buffer(-6).boundary)]))
        gdf_viario_unified = gdf_viario.unary_union

    ## Removendo lotes tipo M
    gdf_lotes = gdf_lotes[gdf_lotes.lo_tp_lote != 'M']

    ## Dissolve o arquivo de lotes por Setor, Quadra, Tipo e talvez Subquadra
    gdf_lotes_dissolved = gdf_lotes.dissolve(by=['lo_setor', 'lo_quadra', 'lo_tp_quad']).reset_index()
    #gdf_lotes_dissolved.geometry = gdf_lotes_dissolved.buffer(0.10).buffer(-0.10)
    gdf_faces = gpd.GeoDataFrame([], geometry=gdf_lotes_dissolved.explode().exterior.droplevel(1).reset_index().drop(columns='index')[0])
    # gdf_faces = gpd.GeoDataFrame([], geometry=gdf_lotes_dissolved.boundary.explode().droplevel(0).reset_index().drop(columns='index')[0])

    ## Calcula-se o centroide de cada segmento de linha de cada quadra
    gdf_pontos_faces = gpd.GeoDataFrame(geometry=gdf_faces.geometry.apply(lambda x: centroide_por_segmento(x)))

    ## Cria um buffer de alguns centimetros para o lote para certificar que o ponto esteja inserido nele
    gdf_lotes_buffered = gpd.GeoDataFrame(gdf_lotes, geometry=gdf_lotes.buffer(.05))

    ## Relaciona cada centroide de segmento de linha ao lote
    gdf_pontos_lotes = gpd.sjoin(gdf_pontos_faces.reset_index().explode().droplevel(0).drop(columns='index'), gdf_lotes_buffered, how='left')
    
    ## Conecta-se o centroide de cada segmento de linha ao segmento de logradouro mais próximo
    gdf_pontos_lotes_copy = gdf_pontos_lotes.copy(deep=True)
    gdf_conexoes = gpd.GeoDataFrame(gdf_pontos_lotes_copy, \
        geometry=gdf_pontos_lotes_copy.geometry.apply(lambda x: \
            LineString(nearest_points(x, gdf_viario_unified))))

    ## Cria Buffer de 50cm do viario
    gdf_viario_buffered = gpd.GeoDataFrame(gdf_viario, geometry=gdf_viario.buffer(.10))

    ## ajuste de indice
    gdf_conexoes = gdf_conexoes.drop(columns=['index_right'])

    ## Cria conexão com a via
    gdf_conexoes_relacionadas = gpd.sjoin(gdf_conexoes, \
        gdf_viario_buffered[['lg_codlog', 'geometry']], how='left', op='intersects')

    ## Relaciona os pontos ao CODLOG
    gdf_pontos_lotes_buffered = gpd.GeoDataFrame(gdf_pontos_lotes, geometry=gdf_pontos_lotes.buffer(0.01))
    gdf_pontos_lote_buffered_relacionados = gpd.sjoin(gdf_pontos_lotes_buffered.drop(columns=['index_right']), gdf_conexoes_relacionadas[['lg_codlog', 'geometry']], how='left')

    ## Segmentos de faces de lotes
    gdf_faces_segmentos = gpd.GeoDataFrame(geometry=gdf_faces.geometry.apply(lambda x: segmentos(x)))
    gdf_faces_segmentos = gdf_faces_segmentos.explode().droplevel(0).reset_index().drop(columns=['index'])  

    ## Certifica que somente os segmentos exteriors estão no conjunto de dados
    gdf_lotes_clip = gdf_lotes_dissolved.buffer(.04).buffer(-0.05).unary_union
    gdf_faces_segmentos = gpd.overlay(gdf_faces_segmentos, gpd.GeoDataFrame(geometry=[gdf_lotes_clip]), how='difference')

    # ## Relaciona-se essa conexão criada ao CODLOG e Lote
    gdf_testadas = gpd.sjoin(gdf_faces_segmentos, gdf_pontos_lote_buffered_relacionados.drop(columns=['index_right']), how='left', op='intersects')

    ## Dissolve-se os segmentos de linha das faces dos lotes em testadas por CODLOG agrupadas por Lote fiscal
    gdf_testadas_dissolved = gdf_testadas.dissolve(by=['lo_setor', 'lo_quadra', 'lo_lote', 'lo_condomi', 'lo_tp_lote', 'lg_codlog'], as_index=False)

    ## Ajustando o SRC
    gdf_testadas_dissolved = gdf_testadas_dissolved.set_crs('epsg:31983')

    ## Exporta os resultados para um arquivo contendo as testadas dos lotes do distrito selecionado
    distrito_prefix = path.replace(f"{folder}/SIRGAS_SHP_LOTES_", '')[0:].replace('.zip', '').replace('_', '-').lower()
    gdf_testadas_dissolved.to_file(f"../resultados/{distrito_prefix}-testadas.gpkg", driver='GPKG')

    ## Grava Log
    print(distrito_prefix)


92-vila-medeiros
08-belem
47-jose-bonifacio
56-pari
44-jardim-helena
20-carrao
80-tatuape
07-bela-vista
05-artur-alvim
89-vila-maria
81-tremembe
50-limao
54-morumbi
53-mooca
42-jaragua
58-pedreira
22-cidade-ademar
45-jardim-paulista
16-campo-grande
10-bras
51-mandaqui
33-iguatemi
30-grajau
77-saude
03-anhanguera
32-moema
26-consolacao
59-penha
21-casa-verde
57-parque-do-carmo
12-butanta
13-cachoeirinha
85-vila-formosa
38-jabaquara
55-parelheiros
64-ponte-rasa
06-barra-funda
63-pirituba
48-lapa
72-sao-lucas
79-socorro
09-bom-retiro
74-sao-miguel
90-vila-mariana
82-tucuruvi
49-liberdade
73-sao-mateus
52-marsilac
02-alto-de-pinheiros
36-itaim-paulista
39-jacana
65-raposo-tavares
28-ermelino-matarazzo
37-itaquera
41-jaguare
88-vila-leopoldina
75-sao-rafael
23-cidade-dutra
86-vila-guilherme
40-jaguara
76-sapopemba
67-rio-pequeno
31-guaianases
95-sao-domingos
60-perdizes
04-aricanduva
17-campo-limpo
35-itaim-bibi
61-perus
93-vila-prudente
14-cambuci
94-vila-sonia
70-santana
25-cidade-tiraden